In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
MODEL
"""
num_classes = 0
# Use a Ladder CNN in addition to the image encoder?
use_cnn = False
# Run the resizing operations after the CNN? (might preserve fine pixel-level details better)
resize_before_cnn = True

# Use LoRA on the mask decoder? (if False, trains the whole decoder)
mask_lora = True
mask_r = 8

# use LoRa on the image encoder transformer?
vit_lora = False
vit_r = 8

# train the image encoder's patch embedding CNN? Highly recommended.
vit_patch_embed = True

"""
TRAINING
"""
run_grad = True

focal_scale = 20
mse_scale = 1
dice_scale = 1
cls_scale = 1

"""
DATA
"""
use_valid = True
cls_ids = None # any
train_size = None # images
valid_size = 20 # images

use_masks = True
grow_masks = False
growth_radius = 15

# train on semantic segmentation or point/box-to-mask?
tasks = ["sem_seg","point","box","cls"]

In [ ]:
import sys
import os
sys.path.append(os.path.relpath("./segment-anything"))
import segment_anything

%cd datasets/

from fine_tune.configs.midas_box import *

%cd ..

In [ ]:
import supervision as sv
from fine_tune.datasets import extract_classes_from_dataset,shrink_dataset_to_size

train_dataset = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset.location}/train",
    annotations_path=f"{dataset.location}/train/_annotations.coco.json",
    force_masks=use_masks
)

if train_size is not None:
    train_dataset = shrink_dataset_to_size(train_dataset,train_size)

if cls_ids is not None:
    print("Selecting classes",[train_dataset.classes[i] for i in cls_ids])
    train_dataset = extract_classes_from_dataset(train_dataset,cls_ids)

if use_valid:
    valid_dataset = sv.DetectionDataset.from_coco(
        images_directory_path=f"{dataset.location}/valid",
        annotations_path=f"{dataset.location}/valid/_annotations.coco.json",
        force_masks=use_masks
    )
else:
    valid_dataset = train_dataset

if valid_size is not None:
    valid_dataset = shrink_dataset_to_size(valid_dataset,valid_size)

if cls_ids is not None:
    valid_dataset = extract_classes_from_dataset(valid_dataset,cls_ids)

In [ ]:
import wandb
wandb.login()

In [ ]:
if use_valid:
    valid_names = set(k.split(".rf")[0] for k in valid_dataset.images.keys())
    train_names = set(k.split(".rf")[0] for k in train_dataset.images.keys())

    # Check that there's no training/valid pollution
    assert len(valid_names.intersection(train_names)) == 0,"There is overlap between the training and validation sets."

In [ ]:
from persam.load import load_predictor
from fine_tune.samed import LoRA_Mask_Decoder,LoRA_Tiny_Image_Encoder
from fine_tune.ladder_cnn import CNN_SAM

predictor = load_predictor("vit_t",num_classes=num_classes)

if mask_lora:
    lora_mask_decoder = LoRA_Mask_Decoder(predictor.model.mask_decoder,r=mask_r)
    mask_decoder = lora_mask_decoder.mask_decoder.cuda()
else:
    mask_decoder = predictor.model.mask_decoder

if vit_lora:
    lora_image_encoder = LoRA_Tiny_Image_Encoder(predictor.model.image_encoder,r=vit_r).cuda()
    image_encoder = lora_image_encoder.image_encoder
else:
    image_encoder = predictor.model.image_encoder

if use_cnn:
    cnn_sam = CNN_SAM(resize_before_cnn=resize_before_cnn).to("cuda")
else:
    cnn_sam = None

In [ ]:
from fine_tune.common import grow_dataset_masks

if grow_masks:
    grow_dataset_masks(train_dataset,growth_radius=growth_radius)
    grow_dataset_masks(valid_dataset,growth_radius=growth_radius)

In [ ]:
from fine_tune.common import SamBoxDataset, SamPointDataset, SamNextMaskDataset, RandomPointDataset, SamSemSegDataset, SamComboDataset, get_max_iou_masks

import torch
device = torch.device('cuda:0')

from torch.utils.data import ConcatDataset

dataset_makers = {
    "sem_seg": lambda ds: SamSemSegDataset(ds,predictor,device),
    "box": lambda ds: SamBoxDataset(ds,predictor,device),
    "point": lambda ds: SamPointDataset(ds,predictor,device,points_per_mask=1),
    "cls": lambda ds: SamMidasBinaryClassification(ds,predictor,device),
}

def prep_datasets(ds):
    datasets = []
    for task in tasks:
        datasets.append(dataset_makers[task](ds))
    return SamComboDataset(datasets)

curr_dataset = prep_datasets(train_dataset)
valid_curr_dataset = prep_datasets(valid_dataset)

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
import numpy as np

# Hyperparameters
initial_lr = 2e-4
weight_decay = 0.1

warmup_steps = 25000
total_steps = 150_000
batch_size = 5 # perform gradient accumulation for this--they do 256 images per batch.
log_period = 200 # a few batches
eval_period = 500
wandb_log_period = 20

lr_decay_steps = [2/3., 0.95]
lr_decay_steps = [int(total_steps * step) for step in lr_decay_steps]

lr_decay_factor = 0.1

combined_params = list(predictor.model.mask_decoder.parameters())
if use_cnn: combined_params += list(cnn_sam.parameters())
if vit_patch_embed: combined_params += list(image_encoder.patch_embed.parameters())
optimizer = optim.AdamW(combined_params, lr=initial_lr, betas=(0.9, 0.999), weight_decay=weight_decay) # criterion = torch.nn.CrossEntropyLoss()

num_params = sum([np.prod(p.size()) for p in combined_params if p.requires_grad])
print(f"Total trainable parameters: {num_params}")

# Learning rate warmup schedule
def warmup_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return 1.0

warmup_scheduler = LambdaLR(optimizer, lr_lambda=warmup_lambda)

# Step-wise learning rate decay schedule
def lr_decay_lambda(current_step):
    if current_step in lr_decay_steps:
        return lr_decay_factor
    return 1.0

lr_decay_scheduler = LambdaLR(optimizer, lr_lambda=lr_decay_lambda)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.functional import threshold, normalize

import cv2
from numpy.random import permutation
from tqdm import tqdm
import json

from persam.persam_f import calculate_sigmoid_focal_loss, calculate_dice_loss

from fine_tune.viz import mask_to_img,clip_together_imgs, show_confusion_matrix, render_prompt


In [ ]:
from PIL import Image
import numpy as np
from persam.persam_f import calculate_iou_loss, calculate_dice_loss, calculate_sigmoid_focal_loss

mse = nn.MSELoss()

# cross entropy loss with logits
bce = nn.CrossEntropyLoss()

def evaluate():
    pred_classes = []
    gt_classes = []

    running_loss = 0.0
    running_count = 0
    for decoder_input, gt_masks, gt_cls_logits, (input_size,original_size), img, (unresized_img,resized_img) in tqdm(valid_curr_dataset):
        with torch.no_grad():
            new_input = {**decoder_input}
            if use_cnn:
                cnn_embedding = cnn_sam(unresized_img,resized_img)[0]
                curr_embedding = decoder_input["image_embeddings"]
                new_input["image_embeddings"] = curr_embedding + cnn_embedding

            low_res_masks, iou_predictions, cls_pred_logits = mask_decoder(**new_input)

            upscaled_masks = predictor.model.postprocess_masks(low_res_masks, input_size, original_size).to(device)
            binary_masks = normalize(threshold(upscaled_masks + 0.1, 0.0, 0)).to(device)[0,:,:,:]

            gt_binary_mask, binary_mask,*_ = get_max_iou_masks(gt_masks,binary_masks)

            max_idx = torch.argmax(iou_predictions)
            binary_mask = binary_masks[max_idx]
            pred_iou = iou_predictions[max_idx]

            pred_logits = cls_pred_logits[0,max_idx]
            assert pred_logits.shape == (mask_decoder.num_classes,),f"pred_logits.shape: {pred_logits.shape}"
            if gt_cls_logits is None:
                cls_loss = 0
            else:
                cls_loss = bce(pred_logits, gt_cls_logits[0].to(device))

                pred_class = torch.argmax(pred_logits)
                gt_class = torch.argmax(gt_cls_logits[0])

                pred_classes.append(pred_class.item())
                gt_classes.append(gt_class.item())

            flat_binary_mask = binary_mask.view(1,-1)
            flat_gt_binary_mask = gt_binary_mask.view(1,-1)

            focal_loss = calculate_sigmoid_focal_loss(flat_binary_mask, flat_gt_binary_mask,should_sigmoid=False)
            dice_loss = calculate_dice_loss(flat_binary_mask, flat_gt_binary_mask, should_sigmoid=False)
            iou_loss = calculate_iou_loss(flat_binary_mask, flat_gt_binary_mask, should_sigmoid=False)

            loss = dice_scale * dice_loss + focal_scale * focal_loss + cls_scale * cls_loss

            running_loss += loss.item()
            running_count += 1
    valid_loss = running_loss/running_count

    wandb.log({
        "valid_loss": valid_loss,
    })

    print(f"VALID - Loss: {valid_loss:.4f}")

    if len(gt_classes) > 0:
        # calculate confusion matrix
        show_confusion_matrix(gt_classes, pred_classes, class_names=valid_dataset.classes)


In [ ]:
run = wandb.init(
    project="sam-fine-tune",
    config={
        "learning_rate": initial_lr,
        "iters": total_steps,
        "batch_size": batch_size,
        "lr_decay_steps": json.dumps(lr_decay_steps),
    })

curr_iters = 0
accumulated_loss = 0

# track running avg of loss
recent_losses = []
recent_mses = []

use_postprocess = True

curr_epoch = 0

# iter through dataset in random order
while curr_iters < total_steps:
    evaluate()
    for i,idx in enumerate(tqdm(permutation(len(curr_dataset)))):
        decoder_input, gt_masks, gt_cls_logits, (input_size,original_size), img, (unresized_img,resized_img) = curr_dataset[idx]
        new_input = {**decoder_input}

        curr_iters += 1

        _,H,W = gt_masks.shape

        if vit_lora:
            sam_embedding = image_encoder(resized_img)[0]
            new_input["image_embeddings"] = sam_embedding
        if use_cnn:
            cnn_embedding = cnn_sam(unresized_img, resized_img).squeeze(0)

            curr_embedding = new_input["image_embeddings"]

            new_input["image_embeddings"] = curr_embedding + cnn_embedding
        

        low_res_masks, iou_predictions, cls_pred_logits = mask_decoder(**new_input)

        high_res_masks = F.interpolate(
                low_res_masks,
                (H, W),
                mode="bilinear",
                align_corners=False,
            )
        high_res_masks = high_res_masks.squeeze(0)
        assert len(high_res_masks.shape) == 3,f"Shape is {high_res_masks.shape}" # 1 or 3, H, W

        # upscaled_masks = F.interpolate(low_res_masks, size=img.shape[:2], mode="bilinear", align_corners=False)
        upscaled_masks = predictor.model.postprocess_masks(low_res_masks, input_size, original_size).to(device)
        binary_masks = normalize(threshold(upscaled_masks, 0.0, 0)).to(device)[0,:,:,:]

        gt_binary_mask, binary_mask, iou, pred_idx = get_max_iou_masks(gt_masks,binary_masks)
        pred_iou = iou_predictions[0,pred_idx]

        pred_mask = upscaled_masks[0,pred_idx] if use_postprocess else high_res_masks[pred_idx]

        pred_logits = cls_pred_logits[0,pred_idx]
        assert pred_logits.shape == (mask_decoder.num_classes,),f"pred_logits.shape: {pred_logits.shape}"

        if gt_cls_logits is None:
            cls_loss = 0
        else:
            cls_loss = bce(pred_logits, gt_cls_logits[0].to(device))

        assert gt_binary_mask.shape == binary_mask.shape,f"Mismatched shapes {gt_binary_mask.shape}, {binary_mask.shape}"
        h,w = gt_binary_mask.shape

        flat_pred_mask = pred_mask.view(1,-1)
        flat_gt_binary_mask = gt_binary_mask.view(1,-1)

        focal_loss = calculate_sigmoid_focal_loss(flat_pred_mask, flat_gt_binary_mask)
        dice_loss = calculate_dice_loss(flat_pred_mask, flat_gt_binary_mask)
        mse_loss = mse(pred_iou, iou)

        loss = dice_scale * dice_loss + mse_scale * mse_loss + focal_scale * focal_loss + cls_scale * cls_loss

        recent_losses += [loss.item()]
        recent_losses = recent_losses[-log_period:]

        if i % wandb_log_period == 0:
            wandb.log({
                "loss": loss.cpu().item(),
                "focal_loss": focal_loss.cpu().item(),
                "dice_loss": dice_loss.cpu().item(),
                "mse_loss": mse_loss.cpu().item(),
                "cls_loss": cls_loss.cpu().item(),
            })

        if i % log_period == 0:
            print(f"Loss: {sum(recent_losses)/len(recent_losses)}")

        if not run_grad: continue
        accumulated_loss += loss
        if curr_iters % batch_size == 0:
            optimizer.zero_grad()
            accumulated_loss /= batch_size
            accumulated_loss.backward()
            optimizer.step()
            accumulated_loss = 0
        
        warmup_scheduler.step(curr_iters)
        lr_decay_scheduler.step(curr_iters)
    
    curr_epoch += 1

In [ ]:
clip_together_imgs(mask_to_img(pred_mask > 0,img),mask_to_img(gt_binary_mask,img))

In [ ]:
from glob import glob
%cd runs/

run_cfgs = glob('cfg_*.json')
highest_run = max([int(cfg.split('_')[1].split('.')[0]) for cfg in run_cfgs])
run_num = highest_run + 1
print(f"Run number: {run_num}")

dataset_path = dataset.location
import os
dataset_name = os.path.basename(dataset_path)

cfg = {
    "dataset_name": dataset_name,
    "use_cnn": use_cnn,
    "resize_before_cnn": resize_before_cnn,
    "grow_masks": grow_masks,
    "mask_lora": mask_lora,
    "mask_r": mask_r,
    "vit_lora": vit_lora,
    "vit_r": vit_r,
    "vit_patch_embed": vit_patch_embed,
    "initial_lr": initial_lr,
    "weight_decay": weight_decay,
    "warmup_steps": warmup_steps,
    "total_steps": total_steps,
    "batch_size": batch_size,
    "lr_decay_steps": lr_decay_steps,
    "lr_decay_factor": lr_decay_factor,
    "run_name":run.name,
    "run_url":run.url,
    "run_grad": run_grad,
    "use_masks": use_masks,
    "dice_scale": dice_scale,
    "focal_scale": focal_scale,
    "cls_scale": cls_scale,
    "num_classes": num_classes,
    "tasks": tasks,
    "use_valid": use_valid,
}

with open(f'cfg_{run_num}.json', 'w') as f:
    json.dump(cfg, f)

torch.save(mask_decoder.state_dict(), f'mask_decoder_{run_num}.pt')
if use_cnn:
    torch.save(cnn_sam.state_dict(), f'ladder_{run_num}.pt')
if mask_lora:
    lora_mask_decoder.save_lora_parameters(f"mask_lora_{run_num}.pt")
if vit_lora:
    lora_image_encoder.save_lora_parameters(f"vit_lora_{run_num}.pt")
if vit_patch_embed:
    torch.save(image_encoder.patch_embed.state_dict(), f"vit_patch_embed_{run_num}.pt")
if num_classes > 0:
    torch.save(mask_decoder.class_tokens.state_dict(), f"class_tokens_{run_num}.pt")
    torch.save(mask_decoder.class_prediction_head.state_dict(), f"class_prediction_head_{run_num}.pt")

%cd ..

In [ ]:
evaluate()

In [ ]:
from random import randint
idx = randint(0,len(valid_curr_dataset)-1)

decoder_input, gt_masks, gt_cls_logits, (input_size,original_size), img, (unresized_img,resized_img) = valid_curr_dataset[idx]
new_input = {**decoder_input}
print("one mask" if len(gt_masks) == 1 else "multiple masks")

with torch.no_grad():
    if use_cnn:
        cnn_embedding = cnn_sam(unresized_img,resized_img)[0]
        curr_embedding = decoder_input["image_embeddings"]
        new_input["image_embeddings"] = curr_embedding + cnn_embedding
    low_res_masks, iou_predictions, cls_pred_logits = mask_decoder(**new_input)

# upscaled_masks = F.interpolate(low_res_masks, size=img.shape[:2], mode="bilinear", align_corners=False)
upscaled_masks = predictor.model.postprocess_masks(low_res_masks, input_size, original_size).to(device)
binary_masks = normalize(threshold(upscaled_masks + 0.1, 0.0, 0)).to(device)[0,:,:,:]

max_idx = torch.argmax(iou_predictions).item()
iou = iou_predictions[max_idx].item()

gt_binary_mask, binary_mask,*_ = get_max_iou_masks(gt_masks,binary_masks)
binary_mask = binary_masks[max_idx]

from PIL import Image
import numpy as np

from persam.persam_f import calculate_iou_loss, calculate_dice_loss, calculate_sigmoid_focal_loss

with torch.no_grad():

    pred_logits = cls_pred_logits[0,max_idx]
    assert pred_logits.shape == (mask_decoder.num_classes,),f"pred_logits.shape: {pred_logits.shape}"
    if gt_cls_logits is None:
        cls_loss = 0
    else:
        cls_loss = bce(pred_logits, gt_cls_logits[0].to(device))

    flat_binary_mask = binary_mask.view(1,-1)
    flat_gt_binary_mask = gt_binary_mask.view(1,-1)

    focal_loss = calculate_sigmoid_focal_loss(flat_binary_mask, flat_gt_binary_mask,should_sigmoid=False)
    dice_loss = calculate_dice_loss(flat_binary_mask, flat_gt_binary_mask, should_sigmoid=False)
    iou_loss = calculate_iou_loss(flat_binary_mask, flat_gt_binary_mask, should_sigmoid=False)
    loss = focal_loss + dice_loss
    print(f"Loss: {loss:.4f}, Focal Loss: {focal_loss:.4f}, Dice Loss: {dice_loss:.4f}, IoU Loss: {iou_loss:.4f}")

print("Predicted IoU:",iou)
print("Predicted class:",torch.argmax(pred_logits).item(),"GT class:",torch.argmax(gt_cls_logits[0]).item())

clip_together_imgs(mask_to_img(binary_mask,img),mask_to_img(gt_binary_mask,img))

In [ ]:
upscaled_masks = F.interpolate(low_res_masks, size=img.shape[:2], mode="bilinear", align_corners=False)
# upscaled_masks = predictor.model.postprocess_masks(low_res_masks, input_size, original_size).to(device)

Image.fromarray(upscaled_masks.cpu().numpy()[0,0] * 255).convert("RGB")

In [ ]:
# upscale low_res_masks to 1024x1024
high_res_masks = F.interpolate(low_res_masks, size=img.shape[:2], mode="bilinear", align_corners=False).detach().cpu().numpy()[0,0]

thresholded = (high_res_masks > 0.5).astype(np.uint8)
Image.fromarray(img * thresholded[...,None]).convert("RGB")

In [ ]:
from random import choice

name,prompt = choice(cls_dataset.prompts)
print(prompt)
render_prompt(name,prompt,train_dataset)

# Segment Everything

In [ ]:
from segment_anything import SamAutomaticMaskGenerator
from segment_anything import automatic_mask_generator
from persam.load import load_predictor
import cv2

model = load_predictor("vit_h").model.cuda()
# model.mask_decoder = mask_decoder

img = cv2.imread("demo-img.png")#list(train_dataset.images.values())[0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# raise NotImplementedError("TODO: fix this")

mask_generator = SamAutomaticMaskGenerator(model,stability_score_thresh=0.1)
print(f"Model threshold: {model.mask_threshold}, offset: {mask_generator.stability_score_offset}")

sam_result = mask_generator.generate(img)
raw_masks = automatic_mask_generator.latest_masks

if len(sam_result) == 0:
    print("No detections found")
    raise NotImplementedError("TODO: fix this")
detections = sv.Detections.from_sam(sam_result=sam_result)

annotator = sv.MaskAnnotator()
# paste each image into the new image
# detections = pred_dataset.annotations[img_name]
tmp_img = annotator.annotate(scene=img,detections=detections)
tmp_img = Image.fromarray(tmp_img)

tmp_img

In [ ]:
from PIL import Image
import numpy as np
print(raw_masks.shape,raw_masks.dtype)
Image.fromarray(raw_masks[0].detach().cpu().numpy().astype(np.uint8) * 255)

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
from fine_tune.samed import LoRA_Mask_Decoder

lora_mask_decoder = LoRA_Mask_Decoder(sam.mask_decoder,r=5)

lora_mask_decoder.load_lora_parameters("lora.pt")
mask_decoder = lora_mask_decoder.mask_decoder.cuda()

In [ ]:
import cv2
img = cv2.imread("demo-img.png")#list(train_dataset.images.values())[0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

masks = mask_generator.generate(img)

masks

# Distance utils

In [ ]:
from supervision.dataset.utils import approximate_mask_with_polygons

from random import choice
rand_img_name = choice(list(train_dataset.images.keys()))

rand_img_dets = train_dataset.annotations[rand_img_name]

biggest_mask = rand_img_dets.mask[rand_img_dets.area.argmax()]

from supervision.dataset.utils import approximate_mask_with_polygons
def dets_to_polygonss(dets):
    polygons = []
    for _,det_mask,*_ in dets:
        polygons.append(approximate_mask_with_polygons(det_mask))
    return polygons

def get_distances(point, polygonss):
    distances = np.zeros(len(polygonss), dtype=np.float32)
    for i,polygons in enumerate(polygonss):
        distances[i] = min([-cv2.pointPolygonTest(polygon, point, True) for polygon in polygons])
    
    distances = np.maximum(distances, 0)
    return distances

def show_closest_distances(point, dets):
    # get the distance of each det
    polygonss = dets_to_polygonss(dets)
    distances = get_distances(point, polygonss)

    # render the masks with their respective distances
    empty_map = np.zeros_like(dets.mask[0], dtype=np.float32)

    for i,(_,det_mask,*_) in enumerate(dets):
        empty_map[det_mask] = distances[i]

    return empty_map

polygons = approximate_mask_with_polygons(biggest_mask)

# make a signed distance map
empty_map = np.zeros(biggest_mask.shape, dtype=np.float32)

for i in range(biggest_mask.shape[0]):
    for j in range(biggest_mask.shape[1]):
        redPoint = (j,i)
        closestDist = cv2.pointPolygonTest(polygons[0], redPoint, True)

        empty_map[i, j] = -closestDist

empty_map = np.maximum(empty_map, 0)

from matplotlib import pyplot as plt

rand_point = (choice(range(biggest_mask.shape[0])), choice(range(biggest_mask.shape[1])))

plt.imshow(show_closest_distances(rand_point, rand_img_dets))

# Progressive mask gen

In [ ]:
import supervision as sv

class LabellingSession:
    def __init__(self,img:np.ndarray):
        self.img = img
        self.sv_dataset = sv.DetectionDataset(
            classes=train_dataset.classes,
            images={
                "test":img
            },
            annotations={
                "test":sv.Detections.empty()
            }
        )
        self.torch_dataset = SamNextMaskDataset(self.sv_dataset,predictor,device)
        self.annotator = sv.MaskAnnotator()
    
    def update_detections(self,detections:sv.Detections):
        self.sv_dataset.annotations["test"] = detections

    def img_to_tensors(self):
        return self.torch_dataset[0]
    
    def show(self):
        return Image.fromarray(self.annotator.annotate(scene=self.img,detections=self.sv_dataset.annotations["test"]))
    
    @torch.no_grad()
    def get_next_mask(self):
        decoder_input, _, (input_size,original_size), img, (unresized_img,resized_img) = self.img_to_tensors()
        H,W,_ = img.shape

        new_input = {**decoder_input}

        if vit_lora:
            sam_embedding = image_encoder(resized_img)[0]
            new_input["image_embeddings"] = sam_embedding

        if use_cnn:
            cnn_embedding = cnn_sam(unresized_img,resized_img)[0]

            curr_embedding = new_input["image_embeddings"]

            new_input["image_embeddings"] = curr_embedding + cnn_embedding

        low_res_masks, iou_predictions = mask_decoder(**new_input)

        high_res_masks = F.interpolate(
                low_res_masks,
                (H, W),
                mode="bilinear",
                align_corners=False,
            )
        high_res_masks = high_res_masks.squeeze(0)
        assert len(high_res_masks.shape) == 3,f"Shape is {high_res_masks.shape}" # 1 or 3, H, W

        assert len(iou_predictions.shape) == 1, f"IoU predictions shape is {iou_predictions.shape}"

        pred_idx = torch.argmax(iou_predictions)
        pred_mask = high_res_masks[pred_idx] > 0

        # new detection
        masks = pred_mask[None, :, :].cpu().numpy()
        class_id = 0
        new_dets = sv.Detections(
            xyxy=sv.Detections.mask_to_xyxy(masks),
            mask=masks,
            class_id=np.array([class_id]),
        )

        self.update_detections(new_dets)


In [ ]:
demo_img = cv2.imread("test.jpg")

session = LabellingSession(demo_img)

In [ ]:
session.get_next_mask()

session.show